<a href="https://colab.research.google.com/github/EmaMule/Computer-Vision/blob/main/CVUSA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import and installing dependencies

In [ ]:
%%capture

# @title Installing dependencies
!pip install tqdm
!pip install pytorch_lightning


In [ ]:
# @title Importing libraries
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler,SequentialSampler,RandomSampler
from torch import nn
import torch
import pytorch_lightning as pl
import pandas as pd
import numpy as np
import random
from torch.utils.data import Dataset, DataLoader, Sampler, random_split
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, TQDMProgressBar, RichProgressBar, ModelPruning
import torch.nn.functional as F
from torchtext.data.metrics import bleu_score
from pytorch_lightning import loggers
from torch.utils.data.sampler import BatchSampler
import warnings
import matplotlib.pyplot as plt
import os
from PIL import Image
from tqdm import tqdm
from google.colab import drive
from torchvision import transforms
import shutil
import csv
import torchvision.models as models

In [ ]:
# @title Mounting drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
#!gdown --id 17W9VEPMneRlb6igtSxa--Xh4fSZs3RS_

In [ ]:
#!unrar x -o+ -v CVUSA_subset.rar | grep -c "^Extracting"

# To be removed (later) REMEMBER: YOU NOW USE POLAR TRANSFORM!!!

In [ ]:
# # @title Unzipping the .rar dataset: code used in creation of dataset
# import os
# from rarfile import RarFile

# rar_file_path = 'drive/MyDrive/CV Project/CVUSA_subset.rar'
# # Open the .rar file
# with RarFile(rar_file_path, 'r') as rar:
#     # Get the list of files in the archive
#     file_list = rar.namelist()

#     # Iterate through the files and extract them
#     for file_name in tqdm(file_list, desc="Extracting", unit="files"):
#         # Extract the file to the current directory
#         rar.extract(file_name, path='drive/MyDrive/CV Project/CVUSA')

In [ ]:
# # @title Code for transforming csv files (originally quite wrong): code used in creation of dataset
# import pandas as pd

# train_csv = "drive/MyDrive/CV Project/CVUSA/train-19zl.csv"
# val_csv = "drive/MyDrive/CV Project/CVUSA/val-19zl.csv"
# train_output_csv = "drive/MyDrive/CV Project/CVUSA/train.csv"
# val_output_csv = "drive/MyDrive/CV Project/CVUSA/val.csv"

# def generate_new_csv(csv_path, out_csv_path):
#   # Read the CSV file
#   df = pd.read_csv(csv_path, header=None)

#   # Rename columns
#   df.rename(columns={0: 'col1', 1: 'col2', 2: 'col3'}, inplace=True)

#   # Change .png to .jpg in col2
#   df['col2'] = df['col2'].str.replace('.png', '.jpg').str.replace('input', '')

#   # Change 'streetview' to 'segmap' and 'input' to 'output' in col3
#   df['col3'] = df['col3'].str.replace('streetview', 'segmap').str.replace('input', 'output')

#   # Add new columns
#   df['col4'] = 'polarmap/' + df['col1'].str.replace('bingmap', 'normal')
#   df['col5'] = 'polarmap/' + df['col3']

#   # Add header
#   header = ["bingmap", "streetview", "segmap", "polarmap/input", "polarmap/segmap"]
#   df.columns = header

#   # Save the modified dataframe to a new CSV file
#   df.to_csv(out_csv_path, index=False)

# generate_new_csv(train_csv, train_output_csv)
# generate_new_csv(val_csv, val_output_csv)


#Dataset and DataModule

In [ ]:
pl.seed_everything(42)

INFO:lightning_fabric.utilities.seed:Seed set to 42


42

In [ ]:
device="cuda" if torch.cuda.is_available() else "cpu"

We need to also possibly add polar and segmap! not done right now because there is a problem with the csv files. Also no test set, should we use validation or split the training and use the current validation as test?

In [ ]:
# @title Dataset definition: without using polar transforms (neither segmentation)
class CVUSADataset(Dataset):
    def __init__(self, data_dir, split = 'train'):

        self.split = split
        self.data = self.load_data(data_dir + f'/{split}.csv')


    def load_data(self, csv_path):
        data = []
        with open(csv_path, 'r') as file:
            csv_reader = csv.reader(file)
            next(csv_reader) #skip header
            for row in csv_reader:
                bingmap = row[3]
                streetview = row[1]
                #segmap = row[2]
                data.append({"streetview": streetview, "bingmap": bingmap})

        return data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        dictionary = self.data[index]
        streetview = dictionary['streetview']
        bingmap = dictionary['bingmap']
        return streetview, bingmap

    def __str__(self):
        return f"CVUSA-Dataset-{self.split}: {len(self.data)} samples"

In [ ]:
# @title Data module definition: without using polar transforms (neither segmentation)
class CVUSADataModule(pl.LightningDataModule):
    def __init__(self, data_dir, batch_size=8, resize = True):

        # Initialize the CustomDataModule

        super(CVUSADataModule, self).__init__()
        self.batch_size = batch_size
        self.data_dir = data_dir
        self.resize = resize
    def setup(self, stage=None):

        # Load the entire dataset
        self.train_dataset = CVUSADataset(data_dir=self.data_dir, split='train')
        print(self.train_dataset)

        #self.test_dataset = CVUSADataset(data_dir=self.data_dir, split='test')
        #print(self.test_dataset)

        self.val_dataset = CVUSADataset(data_dir=self.data_dir, split='val')
        print(self.val_dataset)

    #collate function is useful so we don't overuse RAM, training is a little bit slower tho...
    def collate_fn(self,batch):
        streetview_path, bingmap_path = zip(*batch)
        # Load and transform each image in the batch
        streetview_images_tensor = self.__images_to_tensor(streetview_path)

        bingmap_images_tensor = self.__images_to_tensor(bingmap_path)

        return streetview_images_tensor, bingmap_images_tensor

    #we could add transformations (first of all normalization of the input!)
    def __images_to_tensor(self, paths):
        images = []
        for img_path in paths:
            img = Image.open(os.path.join(self.data_dir, img_path))
            if self.resize:
              img = transforms.Resize((64))(img)

            img_tensor = transforms.ToTensor()(img)
            images.append(img_tensor)

        # Stack the image tensors along the batch dimension
        images_tensor = torch.stack(images)

        return images_tensor



    def train_dataloader(self):
        return DataLoader(self.train_dataset,batch_size=self.batch_size,collate_fn=self.collate_fn,shuffle=True,num_workers=2)

    def val_dataloader(self):
        return DataLoader(self.val_dataset,batch_size=self.batch_size,collate_fn=self.collate_fn,shuffle=False,num_workers=2)

    #def test_dataloader(self):
    #    return DataLoader(self.test_dataset,batch_size=self.batch_size, collate_fn=self.collate_fn,shuffle=True,num_workers=2)

In [ ]:
# @title Creating dataloaders: without using polar transforms (neither segmentation)
data_dir = '/content/drive/MyDrive/CV Project/CVUSA'

data_module = CVUSADataModule(data_dir = data_dir, batch_size = 64)
data_module.setup()

train_loader = data_module.train_dataloader()
val_loader = data_module.val_dataloader()
#test_loader = data_module.test_dataloader()

CVUSA-Dataset-train: 6647 samples
CVUSA-Dataset-val: 2215 samples


#Losses and other utilities

In [ ]:
# # @title TripletLoss implementation
# class TripletLoss(pl.LightningModule):
#     def __init__(self, margin=1.0):
#         super(TripletLoss, self).__init__()
#         self.margin = margin

#     def calc_euclidean(self, x1, x2):
#         return (x1 - x2).pow(2).sum(1)

#     def forward(self, anchor: torch.Tensor, positive: torch.Tensor, negative: torch.Tensor) -> torch.Tensor:
#         distance_positive = self.calc_euclidean(anchor, positive)
#         distance_negative = self.calc_euclidean(anchor, negative)
#         losses = torch.relu(distance_positive - distance_negative + self.margin)

#         return losses.mean()

In [ ]:
# # @title TripletLoss implementation
# class TripletLoss(pl.LightningModule):
#     def __init__(self, margin=1.0):
#         super(TripletLoss, self).__init__()
#         self.margin = margin

#     def forward(self, image_features1, image_features2, k = None):
#         N = len(image_features1)
#         if k is None:
#           k = N-1
#         # Calcolare le distanze Euclidee tra le features
#         distances_per_image1 = torch.cdist(image_features1, image_features2, p=2)  # p=2 per distanza Euclidea
#         distances_per_image2 = distances_per_image1.T  # Per simmetria
#         #in common between the losses:
#         mask = torch.eye(distances_per_image1.size(0), dtype=torch.bool, device = device)
#         distances_diag = torch.masked_select(distances_per_image1, mask) @ torch.eye(n=N, device = device)

#         distances_positive_matrix = distances_diag @ torch.ones(size=(N,k), device = device) #da controllare
#         margin_matrix = torch.ones(size = (N,k), device = device) * self.margin


#         distances_no_diag = torch.masked_select(distances_per_image1, mask.logical_not()).view(distances_per_image1.size(0), -1) #need to see if is NxN-1
#         distances_no_diag, _ = torch.topk(distances_no_diag, k = k, dim = 1, largest = False)
#         loss_matrix = torch.relu(distances_positive_matrix - distances_no_diag + margin_matrix)

#         loss1 = torch.mean(loss_matrix, dim=(0,1))

#         distances_no_diag = torch.masked_select(distances_per_image2, mask.logical_not()).view(distances_per_image2.size(0), -1) #need to see if is NxN-1
#         distances_no_diag, _ = torch.topk(distances_no_diag, k = k, dim = 1, largest = False)
#         loss_matrix = torch.relu(distances_positive_matrix - distances_no_diag + margin_matrix)

#         loss2 = torch.mean(loss_matrix, dim=(0,1))

#         return (loss1 + loss2)/2

In [ ]:
#@title Implementation TripletLoss more stable
class TripletLoss(pl.LightningModule):
    def __init__(self):
        super().__init__()

    def forward(self, grd_global, sat_global, args):
        dist_array = 2.0 - 2.0 * torch.matmul(sat_global, grd_global.T)

        pos_dist = torch.diag(dist_array)
        pair_n = args.train_batch_size * (args.train_batch_size - 1.0)
        triplet_dist_g2s = pos_dist - dist_array
        loss_g2s = torch.sum(torch.log(1.0 + torch.exp(triplet_dist_g2s * args.loss_weight)))/pair_n
        triplet_dist_s2g = torch.unsqueeze(pos_dist, 1) - dist_array
        loss_s2g = torch.sum(torch.log(1.0 + torch.exp(triplet_dist_s2g * args.loss_weight)))/pair_n
        loss = (loss_g2s + loss_s2g) / 2.0

        return loss

In [ ]:
#@title InfoNCE implementation
class InfoNCE(pl.LightningModule):

    def __init__(self, loss_function):
        super().__init__()

        self.loss_function = loss_function #we can use a generic loss function!

    def forward(self, image_features1, image_features2, logit_scale):
        image_features1 = F.normalize(image_features1, dim=-1)
        image_features2 = F.normalize(image_features2, dim=-1)

        logits_per_image1 = logit_scale * image_features1 @ image_features2.T #similarity matrix

        logits_per_image2 = logits_per_image1.T

        labels = torch.arange(len(logits_per_image1), dtype=torch.long, device = device)

        loss = (self.loss_function(logits_per_image1, labels) + self.loss_function(logits_per_image2, labels))/2

        return loss

In [ ]:
# @title Top-K Rank Accuracy: takes embeddings in input

def top_k_rank_accuracy(emb1, emb2, k=1):
    if k > len(emb1) :
      return 0.0 #might happen at the end of the dataset (batch less then the chosen one)
    # Calculate cosine similarity
    correct_in_top_k = 0
    for index, elem in enumerate(emb1):
        cosine_sim = F.cosine_similarity(elem, emb2, dim=1)

        # Find the rank K similarity
        top_k_similarities, top_k_indices = torch.topk(cosine_sim, k, largest=True)

        correct_in_top_k += index in top_k_indices.tolist()

    accuracy = correct_in_top_k / len(emb1)
    return accuracy

In [ ]:
# @title Generation of negatives: modularized so that we can experiment with a lot of versions!
def generate_negatives(positives):
  shuffled_indices = torch.randperm(positives.size(0))
  negatives = positives[shuffled_indices]

  return negatives

# Training Architectures

In [ ]:
class ResNet50Branch(pl.LightningModule):
    def __init__(self):
        super(ResNet50Branch, self).__init__()
        self.resnet50 = models.resnet50(pretrained=True)
        # Modify the last layer for your specific task
        self.resnet50.fc = torch.nn.Linear(self.resnet50.fc.in_features, 512)

    def forward(self, x):
        return self.resnet50(x)

class DualResNet50Model(pl.LightningModule):
    def __init__(self, margin = 1.0):
        super(DualResNet50Model, self).__init__()
        self.branch1 = ResNet50Branch()
        self.branch2 = ResNet50Branch()

        self.loss = InfoNCE(loss_function=nn.CrossEntropyLoss())
        #self.loss = TripletLoss(margin = 0.5)
    def forward(self, x1, x2):
        out1 = self.branch1(x1)
        out2 = self.branch2(x2)
        return out1, out2

    def training_step(self, batch, batch_idx):
        streetview, bingmap = batch
        out1, out2 = self(streetview, bingmap)

        #1 indicates necessary for loss 1, 2 for loss 2. we take the average.
        """anchor_embed1 = out1
        positive_embed1 = out2

        negative_embed1 = generate_negatives(positive_embed1)
        loss1 = self.loss(anchor_embed1, positive_embed1, negative_embed1) #first loss

        loss2 = 0.0
        if self.double_loss:
          anchor_embed2 = out2
          positive_embed2 = out1

          negative_embed2 = generate_negatives(positive_embed2)
          loss2 = self.loss(anchor_embed2, positive_embed2, negative_embed2)

        loss = loss1 + loss2"""

        loss = self.loss(out1, out2, logit_scale = 3.0)

        accuracy_top_1 = top_k_rank_accuracy(out1, out2, k = 1)

        self.log('train_top1', accuracy_top_1, on_step=True, on_epoch=True, prog_bar=True)
        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True)
        return loss


    def validation_step(self, batch, batch_idx):
        streetview, bingmap = batch
        out1, out2 = self(streetview, bingmap)

        accuracy_top_1 = top_k_rank_accuracy(out1, out2, k = 1)
        accuracy_top_3 = top_k_rank_accuracy(out1, out2, k = 3)
        accuracy_top_10 = top_k_rank_accuracy(out1, out2, k = 10)

        self.log('val_top1', accuracy_top_1, on_step=True, on_epoch=True, prog_bar=True)
        self.log('val_top3', accuracy_top_3, on_step=True, on_epoch=True, prog_bar=True)
        self.log('val_top10', accuracy_top_10, on_step=True, on_epoch=True, prog_bar=True)

        return accuracy_top_1, accuracy_top_3, accuracy_top_10


    def test_step(self, batch, batch_idx):
        streetview, bingmap = batch
        out1, out2 = self(streetview, bingmap)

        accuracy_top_1 = top_k_rank_accuracy(out1, out2, k = 1)
        accuracy_top_3 = top_k_rank_accuracy(out1, out2, k = 3)
        accuracy_top_10 = top_k_rank_accuracy(out1, out2, k = 10)

        self.log('test_top1', accuracy_top_1, on_step=True, on_epoch=True, prog_bar=True)
        self.log('test_top3', accuracy_top_3, on_step=True, on_epoch=True, prog_bar=True)
        self.log('test_top10', accuracy_top_10, on_step=True, on_epoch=True, prog_bar=True)

        return accuracy_top_1, accuracy_top_3, accuracy_top_10

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-3)

# Instantiate the model
model = DualResNet50Model(margin = 1.0)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
trainer = pl.Trainer(
    max_epochs=30,
    devices=1,
    callbacks=[RichProgressBar()],
    log_every_n_steps=3
)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(model=model, train_dataloaders=train_loader, val_dataloaders=val_loader)

Epoch 29/29 ━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━ 32/104 0:00:28 • 0:01:02 1.16it/s v_num: 12.000 train_top1_step:    
                                                                                 0.953 train_loss_step: 1.692      
                                                                                 val_top1_step: 0.718              
                                                                                 val_top3_step: 1.000              
                                                                                 val_top10_step: 1.000             
                                                                                 val_top1_epoch: 0.637             
                                                                                 val_top3_epoch: 0.904             
                                                                                 val_top10_epoch: 0.986            
                                                                                 train_top1_epoch: 0.931           
                                                                                 train_loss_epoch: 1.685           

In [ ]:
trainer.test(dataloaders=val_loader)

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:145: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.test(ckpt_path='best')` to use the best model or `.test(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/lightning_logs/version_9/checkpoints/epoch=0-step=52.ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from the checkpoint at /content/lightning_logs/version_9/checkpoints/epoch=0-step=52.ckpt


Output()

AssertionError: 

In [ ]:
del model

#Depth estimation

In [ ]:
from transformers import pipeline

checkpoint = "vinvino02/glpn-nyu"
depth_estimator = pipeline("depth-estimation", model=checkpoint)

In [ ]:
from PIL import Image
import requests

url = "https://unsplash.com/photos/HwBAsSbPBDU/download?ixid=MnwxMjA3fDB8MXxzZWFyY2h8MzR8fGNhciUyMGluJTIwdGhlJTIwc3RyZWV0fGVufDB8MHx8fDE2Nzg5MDEwODg&force=true&w=640"
image = Image.open(requests.get(url, stream=True).raw)
image

In [ ]:
predictions = depth_estimator(image)

In [ ]:
predictions["depth"]

In [ ]:
from transformers import AutoImageProcessor, AutoModelForDepthEstimation

checkpoint = "vinvino02/glpn-nyu"

image_processor = AutoImageProcessor.from_pretrained(checkpoint)
model = AutoModelForDepthEstimation.from_pretrained(checkpoint)

In [ ]:
image = Image.open("/content/drive/MyDrive/CV Project/CVUSA/streetview/0000052.jpg")
with torch.no_grad():
      pixel_values = image_processor(image, return_tensors="pt").pixel_values
      outputs = model(pixel_values)
      predicted_depth = outputs.predicted_depth

In [ ]:
import numpy as np

# interpolate to original size
prediction = torch.nn.functional.interpolate(
    predicted_depth.unsqueeze(1),
    size=image.size[::-1],
    mode="bicubic",
    align_corners=False,
).squeeze()
output = prediction.numpy()

formatted = (output * 255 / np.max(output)).astype("uint8")
depth = Image.fromarray(formatted)
depth

In [ ]:
#road
#building
#low vegetation
#trees
#car
#clutter

In [ ]:
#negative sampling basato sulla similarità ovvero computo la similarità di un sample con tutti gli altri e uso quelli più vicini come negativi

In [ ]:
#soft margin triplet loss and weighted soft margin triplet loss